In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import theano.tensor as T
import numpy as np

In [4]:
from wordvecs import wordVectors

wordvectors = wordVectors(fname="../enwiki-20141208-pages-articles-multistream-links-output4.bin", negvectors=True)

In [5]:
len(wordvectors.vectors)

6091842

In [6]:
l_in = InputLayer((3,))

In [7]:
W = np.arange(3*5).reshape((3,5)).astype('float32')

In [8]:
l1 = EmbeddingLayer(l_in, input_size=3, output_size=5, W=W)

In [9]:
x = T.imatrix()

In [10]:
output = get_output(l1, x)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [11]:
f = theano.function([x], output)

In [12]:
x_test = np.array([[0,2],[1,2]]).astype('int32')

In [13]:
f(x_test)

array([[[  0.,   1.,   2.,   3.,   4.],
        [ 10.,  11.,  12.,  13.,  14.]],

       [[  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.]]], dtype=float32)